# PR Reviewer Agent
The core functionality of a reflection agent is to perform evaluation and refinement of the output through a feedback loop. In our first notebook, we implemented a (Press Release) PR generator agent responsible for drafting PRs. In this notebook, we are implementing a PR reviewer agent that critically evaluates the generated output, identifying potential errors, inconsistencies, or areas for improvement. During this phase, the agent acts as its own critic, scrutinizing the quality, accuracy, and appropriateness of its response.

The following diagram highlights the PR reviewer agent in the Agentic AI architecture for the PR generation workflow:

<img src="../imgs/pr-reflection-agent-reviewer.png" width="600">

Let's get started!

First we restore variables from the previous notebook

In [ ]:
%store -r

## Create a PR Reviewer Agent using Amazon Bedrock Agent
In this section we declare global variables that will act as helpers during entire notebook.

In [ ]:
import boto3

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'us.amazon.nova-pro-v1:0'
]

## Importing helper functions
In the following section, we're adding bedrock_agent_helper.py and knowledge_base_helper on Python path, so the files can be recognized and their functionalities can be invoked.

In general, the helper functions handle common tasks including agent creation, Knowledge Bases for Bedrock creation, and accessing data in S3.

In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

## PR Reviewer Agent
Create the PR reviewer agent to provide a thorough review and provide feedback for the given PR article. 

We will provide the following instructions to our agent:
```
You are an AI assistant agent that performs a review for a given PR article. 
As a PR article reviewer, you are given the specific guidelines to evaluate the quality of the document.

1. Write in clear, crisp sentences." - long or confusing sentences as "clunky" when they're difficult for readers to understand.

Here are the tips on to identify unclear sentences:

- Sentences longer than 40 words or spanning more than two lines
- Sentences that require multiple readings to understand
- Sentences with repeated words, especially "and" and "to"

2. Be specific. Don't leave anything open to interpretation or leave your reader guessing. 

Here are some clues your sentence isn't specific enough: 
-  it contains qualitative or subjective adjectives and adverbs (like "might," "often," or "may be"), 
- it leaves the reader asking questions.

Here are some ways to fix this problem: 
- clarify words or phrases that could be interpreted differently by different people, 
- use quantitative adjectives, like "3X faster" or "50% more time" where possible, and/or 

3. Write for a non-technical audience. Anyone should be able to understand your message without additional background information. Readers will often be left with questions if you've included concepts or undefined acronyms.

Here are some clues your sentence might be too technical: 
- your reader is left with questions, 
- it contains abbreviations or acronyms that aren't defined, and/or 
- it describes technology that an average person would not understand.

Here are some ways to fix this problem: 
3a. Include a short description of the show;
3b. define acronyms if used in the article

4. Proactively explain what makes a new show/movie interesting from similar existing show/movie. If there is any chance a reader might ask, "Isn't this movie similar to X?" then you need to address it. Avoid simply listing plots of the new movie.

Here are some ways to fix this problem:
- If there is an existing movie/show with similar plots as the new movie, address how they are different.

5. Connect the dots in a story.  You should make it clear enough how plots are related.

Here are some clues you're not connecting the dots: 
- your reader doesn't understand the plots you're writing about and/or 

Here is a way to fix this problem: 
Similar to unfolding the story, Have someone who hasn't heard of the story read the passage and ask them if it's clear to them and how everything is related.

6. Be consistent in names and descriptions. Don't confuse your reader by switching between different names or introducing additional characters near the end of your PR. Stick to one name and don't stray from the main plots.
```

In [ ]:
agent_description = """You are a professional PR article reviewer agent. You review PR articles for an upcoming release of new movie/shows and provides critiques and suggestions on how to improve the given article."""
agent_instruction = """You are an AI assistant agent that performs a review for a given PR article and provide any feedback to improve the quality. 
As a PR article reviewer, you are given the specific guidelines to evaluate the quality of the document.

1. Write in clear, crisp sentences." - long or confusing sentences as "clunky" when they're difficult for readers to understand.

Here are the tips on to identify unclear sentences:

- Sentences longer than 40 words or spanning more than two lines
- Sentences that require multiple readings to understand
- Sentences with repeated words, especially "and" and "to"

2. Be specific. Don't leave anything open to interpretation or leave your reader guessing. 

Here are some clues your sentence isn't specific enough: 
-  it contains qualitative or subjective adjectives and adverbs (like "might," "often," or "may be"), 
- it leaves the reader asking questions.

Here are some ways to fix this problem: 
- clarify words or phrases that could be interpreted differently by different people, 
- use quantitative adjectives, like "3X faster" or "50% more time" where possible, and/or 

3. Write for a non-technical audience. Anyone should be able to understand your message without additional background information. Readers will often be left with questions if you've included concepts or undefined acronyms.

Here are some clues your sentence might be too technical: 
- your reader is left with questions, 
- it contains abbreviations or acronyms that aren't defined, and/or 
- it describes technology that an average person would not understand.

Here are some ways to fix this problem: 
3a. Include a short description of the show;
3b. define acronyms if used in the article

4. Proactively explain what makes a new show/movie interesting from similar existing show/movie. If there is any chance a reader might ask, "Isn't this movie similar to X?" then you need to address it. Avoid simply listing plots of the new movie.

Here are some ways to fix this problem:
- If there is an existing movie/show with similar plots as the new movie, address how they are different.

5. Connect the dots in a story.  You should make it clear enough how plots are related.

Here are some clues you're not connecting the dots: 
- your reader doesn't understand the plots you're writing about and/or 

Here is a way to fix this problem: 
Similar to unfolding the story, Have someone who hasn't heard of the story read the passage and ask them if it's clear to them and how everything is related.

6. Be consistent in names and descriptions. Don't confuse your reader by switching between different names or introducing additional characters near the end of your PR. Stick to one name and don't stray from the main plots."""


In [ ]:
pr_reviewer_agent_name = "pr_reviewer_agent"
pr_reviewer_agent = agents.create_agent(
    pr_reviewer_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False
)

pr_reviewer_agent


In [ ]:
pr_reviewer_agent

Prepare the agent so that it can be invoked.

In [ ]:
agents.prepare(pr_reviewer_agent_name)

## Testing the Agent
At this point, you have successfully created a PR reviewer agent with given instructions. 
Next, we'll test the agent independently to verify that the agent follows the directions and works as intended. 
To do so we will use the default alias created: `TSTALIASID` which allows you to invoke a draft version of your agent

In [ ]:
%%time
pr_reviewer_response = agents.invoke(
    f"""Given the following PR article:

{pr_draft_article}
    
Provide a thorough review and recommendation on how to improve the article.
""", 
    pr_reviewer_agent[0], enable_trace=True,
    trace_level = "all"
)

In [ ]:
print(pr_reviewer_response)

## Create an Alias
We've just completed a test query against the PR reviewer agent using the default alias `TSTALIASID`. 

The default Alias is a quick way to test out agents before integrating it into your application.
When creating a multi-agent collaboration, it's required to create an Alias explicitly so that it can be used by other agents. This is to ensure the agent is tested, validated, and the functionality is as expected.
Since we've tested and validated our agent, let's now create an alias for it:

In [ ]:
pr_reviewer_alias_id, pr_reviewer_agent_alias_arn = agents.create_agent_alias(
    pr_reviewer_agent[0], 'v1'
)

## Saving information
Let's store the environment variables to be used in other labs.

In [ ]:
pr_reviewer_agent_arn = agents.get_agent_arn_by_name(pr_reviewer_agent_name)
pr_reviewer_agent_id = pr_reviewer_agent[0]

%store pr_reviewer_agent_arn
%store pr_reviewer_agent_alias_arn
%store pr_reviewer_alias_id
%store pr_reviewer_agent_name
%store pr_reviewer_agent_id